In [158]:
# IMPORT PACKAGES
import pandas as pd
import time
import requests

In [159]:
def func(arg1: str, ag2: dict) -> tuple:
    ''' This is what the function does...
    
    Args:
        ag1 (type): this is what it is
    
    Returns:
        (tuple): gooble dee stuff.
    '''
    # step 1

    # step 2

    # step 3

    # return

In [ ]:
if __name__ == "__main__":
    # step 1

    # step 2

    # step 3

    
    
# played with the lasso code below to select a final set of 36 features
    lasso_feats =  ['covar_btc_covar_trades_t_ma_tm8640',
                    'covar_btc_covar_trades_t_min_tm30',
                    'covar_btc_covar_trades_t_min_tm60',
                    'covar_btc_covar_volume_t_min_tm4320',
                    'covar_btc_p_log_t',
                    'covar_btc_r_1min_ema_tm360',
                    'covar_btc_r_5min_skew_tm720',
                    'covar_btc_r_tm1440',
                    'covar_btc_r_tm15',
                    'covar_btc_r_tm86400',
                    'covar_btc_rsi_tm720',
                    'covar_eth_covar_trades_t_max_tm4320',
                    'covar_eth_covar_trades_t_min_tm20',
                    'covar_eth_covar_trades_t_vol_tm60',
                    'covar_eth_covar_volume_t_ma_tm60',
                    'covar_eth_covar_volume_t_ma_tm8640',
                    'covar_eth_covar_volume_t_max_tm4320',
                    'covar_eth_covar_volume_t_max_tm720',
                    'covar_eth_covar_volume_t_sum_tm20',
                    'covar_eth_covar_volume_t_sum_tm60',
                    'covar_eth_covar_volume_t_vol_tm360',
                    'covar_eth_p_t',
                    'covar_eth_r_1min_ema_tm60',
                    'covar_eth_r_1min_ma_tm360',
                    'covar_eth_r_1min_vol_tm20',
                    'covar_eth_r_1min_vol_tm5',
                    'covar_eth_r_5min_ma_tm20160',
                    'covar_eth_r_5min_ma_tm60',
                    'covar_eth_r_5min_min_tm30',
                    'covar_eth_r_5min_min_tm720',
                    'covar_eth_r_cummax_t',
                    'covar_eth_r_cummin_t',
                    'covar_eth_r_tm120',
                    'covar_eth_r_tm1440',
                    'covar_eth_r_tm40320',
                    'covar_eth_volume_t']

In [ ]:
# for year in [2016, 2017, 2018, 2019, 2020, 2021]:
#     lasso_df = df[df.date.dt.year == year].copy()

#     # set lasso parameters
#     rhs_feats = list(included_feats)
#     lhs_col   = 'y_btc_eth_diff_r_tp5_tp370'
#     alpha     = 0.001

#     # scale RHS
#     y = lasso_df[lhs_col].values
#     X = lasso_df[rhs_feats].values
#     X_scaled = StandardScaler().fit_transform(X)

#     # fit
#     model = Lasso(alpha=alpha, fit_intercept=False)
#     model.fit(X_scaled, y)

#     # show selected feats
#     lasso_feats = [rhs_feats[i] for i in np.nonzero(model.coef_)[0]]
#     print(year)
#     print(lasso_feats)

In [ ]:
# TODO ONCE DONE WITH THIS; TURN INTO PYTHON SCRIPT
# TODO MOVE ANY FUNCTIONS THAT I MAY USE ELSEWHERE TO COMMON FOLDER SHARED ACROSS PROJECTS THAT I JUST IMPORT WHEN NEEDED